In [18]:
##IMPORTS
from argparse import ArgumentParser
import csv
import os, glob, sys
import pandas as pd

In [19]:
# Tabla Diferencias. Suma por Comitente y moneda de liquidación "Dif. DÃ­a" (fila N)

#Leemos la tabla sacada arriba con pandas
file = "diferencias.csv"
df = pd.read_csv(file,skiprows=1,quoting=csv.QUOTE_NONE,delimiter=';',decimal=",")
file2 = "Productos por moneda.xlsx"
df2 = pd.read_excel(file2)
mergedStuff = pd.merge(df, df2, on=['Producto'], how='inner')
#Sumamos por comitenente y moneda
df_diferencias = mergedStuff.groupby(['Comitente','Moneda de liquidación '],as_index=False)[['Dif. Día']].sum()

In [20]:
# Tabla Primas. Suma por Comitente y moneda de liquidación "Primas" (fila L)

file3 = "primas.csv"
df = pd.read_csv(file3,skiprows=1,quoting=csv.QUOTE_NONE,delimiter=';',decimal=",")
file2 = "Productos por moneda.xlsx"
df2 = pd.read_excel(file2)
mergedStuff = pd.merge(df, df2, on=['Producto'], how='inner')
#Sumamos por comitenente y moneda
df_primas = mergedStuff.groupby(['Comitente','Moneda de liquidación '],as_index=False)[['Primas']].sum()

In [21]:
#Leer gvRegistryAccount.xls. Archivo de AP5
#Skipeamos 1 linea porque aune siempre tiene la 1linea vacia

file6 = "gvRegistryAccount.xls"
df_ap5 = pd.read_excel(file6)
df_ap5_slim = df_ap5[['Cuenta', 'Comitente CVSA']]

In [22]:
#Leer export - 2021-07-23T152241.834.xlsx. Archivo de AUNE
#Skipeamos 1 linea porque aune siempre tiene la 1linea vacia

file4 = "export - 2021-07-28T153847.268.xlsx"
df_aune = pd.read_excel(file4, skiprows=1)
#Drop la ultima fila, ya que es el total y no sirve
df_aune.drop(df_aune.tail(1).index,inplace=True)
#Extraemos el nro entre corchetes de la co Cuenta para compararla desp
df_aune['Comitente CVSA'] =  df_aune.Cuenta.str.extract('.*\[(.*)\].*')
#Pasamos CVSA a float para poder hacer el merge
df_aune["Comitente CVSA"] = pd.to_numeric(df_aune["Comitente CVSA"])
#Valor de USD Garantia ROFEX. Ver como obtenerlo automaticamente no hardcodeado
USD_Rofex = 96.5900
df_aune['Aune Total Pesificado'] = df_aune['Total']*USD_Rofex
# Solo dejamos las dos columnas que necesitamos
df_aune_slim = df_aune[['Comitente CVSA', 'Aune Total Pesificado']]
df_aune_slim

,Comitente CVSA,Aune Total Pesificado
0,10082,-620107.800
1,10084,-85965.100
2,10121,-44431.400
3,10122,-24147.500
4,8059,-833552.382
5,8946,-39312.130
6,9061,-212304.820
7,9593,-126291.425
8,9706,-109533.060
9,9935,-35738.300


In [23]:
#Merge Diferencias y Primas por Comitente y Moneda.
mergedStuff = pd.merge(df_diferencias, df_primas, on=['Comitente','Moneda de liquidación '], how='inner')
#Cambiamos el nombre
mergedStuff = mergedStuff.rename(columns={"Comitente": "Cuenta"})
mergedStuff

,Cuenta,Moneda de liquidación,Dif. Día,Primas
0,11909,ARS,20000.000000,0.0
1,11909,USDL,-28992.000000,-193280.0
2,17629,USDL,0.000000,0.0
3,53487,USDL,17395.200000,359500.8
4,54251,USDL,0.000000,0.0
5,80722,ARS,17060.000039,0.0
6,101567,ARS,-108055.000000,0.0
7,102123,ARS,25000.000000,0.0
8,140336,USDL,40588.800000,268659.2
9,140338,USDL,-4832.000000,190380.8


In [24]:
#Correlacionamos los comitenes de AP5 a su numero en AUNE
mergedStuff3 = pd.merge(mergedStuff,df_ap5_slim, on=['Cuenta'], how='inner')
mergedStuff3

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA
0,11909,ARS,20000.000000,0.0,3122.0
1,11909,USDL,-28992.000000,-193280.0,3122.0
2,17629,USDL,0.000000,0.0,3192.0
3,53487,USDL,17395.200000,359500.8,3394.0
4,54251,USDL,0.000000,0.0,8946.0
5,80722,ARS,17060.000039,0.0,9732.0
6,101567,ARS,-108055.000000,0.0,9601.0
7,102123,ARS,25000.000000,0.0,9217.0
8,140336,USDL,40588.800000,268659.2,10082.0
9,140338,USDL,-4832.000000,190380.8,10084.0


In [25]:
#Separamos la tabla entre ARS y USDL
mergedStuff3_ars = mergedStuff3[mergedStuff3['Moneda de liquidación '] == 'ARS']
mergedStuff3_usdl = mergedStuff3[mergedStuff3['Moneda de liquidación '] == 'USDL']

In [26]:
#Merge tabla de USDL coon aune-slim
mergedStuff4 = pd.merge(mergedStuff3_usdl,df_aune_slim, on=['Comitente CVSA'], how='outer')
#Llenamos los NaN con 0
mergedStuff4 = mergedStuff4.fillna(0)

In [27]:
# Hacemos el total
mergedStuff4['Super Total'] = mergedStuff4['Dif. Día']+mergedStuff4['Primas']+mergedStuff4['Aune Total Pesificado']
mergedStuff4

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA,Aune Total Pesificado,Super Total
0,11909.0,USDL,-28992.0,-193280.0,3122.0,0.000,-222272.000
1,17629.0,USDL,0.0,0.0,3192.0,0.000,0.000
2,53487.0,USDL,17395.2,359500.8,3394.0,0.000,376896.000
3,54251.0,USDL,0.0,0.0,8946.0,-39312.130,-39312.130
4,140336.0,USDL,40588.8,268659.2,10082.0,-620107.800,-310859.800
5,140338.0,USDL,-4832.0,190380.8,10084.0,-85965.100,99583.700
6,143317.0,USDL,0.0,-53152.0,10127.0,0.000,-53152.000
7,143515.0,USDL,-43488.0,0.0,10121.0,-44431.400,-87919.400
8,143516.0,USDL,43488.0,0.0,10122.0,-24147.500,19340.500
9,0.0,0,0.0,0.0,8059.0,-833552.382,-833552.382


In [28]:
#Hacemos 0 los positivos del Super Total
for i in mergedStuff4.iloc[:,mergedStuff4.columns.get_loc("Super Total"):mergedStuff4.columns.get_loc("Super Total")+4]<0:
    for index, j in enumerate(mergedStuff4[i]):
        if j>0:
            mergedStuff4.at[index, i] = 0
mergedStuff4

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA,Aune Total Pesificado,Super Total
0,11909.0,USDL,-28992.0,-193280.0,3122.0,0.000,-222272.000
1,17629.0,USDL,0.0,0.0,3192.0,0.000,0.000
2,53487.0,USDL,17395.2,359500.8,3394.0,0.000,0.000
3,54251.0,USDL,0.0,0.0,8946.0,-39312.130,-39312.130
4,140336.0,USDL,40588.8,268659.2,10082.0,-620107.800,-310859.800
5,140338.0,USDL,-4832.0,190380.8,10084.0,-85965.100,0.000
6,143317.0,USDL,0.0,-53152.0,10127.0,0.000,-53152.000
7,143515.0,USDL,-43488.0,0.0,10121.0,-44431.400,-87919.400
8,143516.0,USDL,43488.0,0.0,10122.0,-24147.500,0.000
9,0.0,0,0.0,0.0,8059.0,-833552.382,-833552.382


In [29]:
#Agarramos los ARS de Diferencias y Primas
mergedStuff3_ars = mergedStuff3_ars.rename(columns={"Dif. Día": "Dif. Día ars", "Primas": "Primas ars"})
mergedStuff3_ars = mergedStuff3_ars[['Comitente CVSA', 'Dif. Día ars', 'Primas ars']]
mergedStuff3_ars

,Comitente CVSA,Dif. Día ars,Primas ars
0,3122.0,20000.000000,0.0
5,9732.0,17060.000039,0.0
6,9601.0,-108055.000000,0.0
7,9217.0,25000.000000,0.0
10,9719.0,6250.000000,0.0


In [30]:
#Sumamos los ARS en Dif dia Y primas
mergedStuff5 = pd.merge(mergedStuff3_ars, mergedStuff4, on=['Comitente CVSA'], how='outer')
mergedStuff5 = mergedStuff5.fillna(0)
mergedStuff5

,Comitente CVSA,Dif. Día ars,Primas ars,Cuenta,Moneda de liquidación,Dif. Día,Primas,Aune Total Pesificado,Super Total
0,3122.0,20000.000000,0.0,11909.0,USDL,-28992.0,-193280.0,0.000,-222272.000
1,9732.0,17060.000039,0.0,0.0,0,0.0,0.0,0.000,0.000
2,9601.0,-108055.000000,0.0,0.0,0,0.0,0.0,0.000,0.000
3,9217.0,25000.000000,0.0,0.0,0,0.0,0.0,0.000,0.000
4,9719.0,6250.000000,0.0,0.0,0,0.0,0.0,0.000,0.000
5,3192.0,0.000000,0.0,17629.0,USDL,0.0,0.0,0.000,0.000
6,3394.0,0.000000,0.0,53487.0,USDL,17395.2,359500.8,0.000,0.000
7,8946.0,0.000000,0.0,54251.0,USDL,0.0,0.0,-39312.130,-39312.130
8,10082.0,0.000000,0.0,140336.0,USDL,40588.8,268659.2,-620107.800,-310859.800
9,10084.0,0.000000,0.0,140338.0,USDL,-4832.0,190380.8,-85965.100,0.000


In [31]:
#Sumamos a Super Total los Dif. Día ars	Primas ars
mergedStuff5['Super Total'] = mergedStuff5['Dif. Día ars'] + mergedStuff5['Primas ars'] + mergedStuff5['Super Total']

mergedStuff5

,Comitente CVSA,Dif. Día ars,Primas ars,Cuenta,Moneda de liquidación,Dif. Día,Primas,Aune Total Pesificado,Super Total
0,3122.0,20000.000000,0.0,11909.0,USDL,-28992.0,-193280.0,0.000,-202272.000000
1,9732.0,17060.000039,0.0,0.0,0,0.0,0.0,0.000,17060.000039
2,9601.0,-108055.000000,0.0,0.0,0,0.0,0.0,0.000,-108055.000000
3,9217.0,25000.000000,0.0,0.0,0,0.0,0.0,0.000,25000.000000
4,9719.0,6250.000000,0.0,0.0,0,0.0,0.0,0.000,6250.000000
5,3192.0,0.000000,0.0,17629.0,USDL,0.0,0.0,0.000,0.000000
6,3394.0,0.000000,0.0,53487.0,USDL,17395.2,359500.8,0.000,0.000000
7,8946.0,0.000000,0.0,54251.0,USDL,0.0,0.0,-39312.130,-39312.130000
8,10082.0,0.000000,0.0,140336.0,USDL,40588.8,268659.2,-620107.800,-310859.800000
9,10084.0,0.000000,0.0,140338.0,USDL,-4832.0,190380.8,-85965.100,0.000000


In [32]:
df_final = mergedStuff5[['Comitente CVSA','Super Total']]
df_final

,Comitente CVSA,Super Total
0,3122.0,-202272.000000
1,9732.0,17060.000039
2,9601.0,-108055.000000
3,9217.0,25000.000000
4,9719.0,6250.000000
5,3192.0,0.000000
6,3394.0,0.000000
7,8946.0,-39312.130000
8,10082.0,-310859.800000
9,10084.0,0.000000


In [34]:
df_final['Super Total'].sum()

-2070680.3169608566